In [4]:
!pip install tensorflow

   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/376.0 MB 6.7 MB/s eta 0:00:56
    --------------------------------------- 7.3/376.0 MB 25.2 MB/s eta 0:00:15
   - -------------------------------------- 16.5/376.0 MB 34.7 MB/s eta 0:00:11
   --- ------------------------------------ 33.6/376.0 MB 46.3 MB/s eta 0:00:08
   ---- ----------------------------------- 45.9/376.0 MB 48.6 MB/s eta 0:00:07
   ------ --------------------------------- 57.4/376.0 MB 50.1 MB/s eta 0:00:07
   ------- -------------------------------- 67.6/376.0 MB 49.6 MB/s eta 0:00:07
   -------- ------------------------------- 77.9/376.0 MB 49.2 MB/s eta 0:00:07
   --------- ------------------------------ 86.8/376.0 MB 48.6 MB/s eta 0:00:06
   ---------- ----------------------------- 96.2/376.0 MB 48.4 MB/s eta 0:00:06
   ---------- ---------------------------- 105.9/376.0 MB 48.0 MB/s eta 0:00:06
   ----------- --------------------------- 115.3/376

# Bước 1: Import thư viện và tiền xử lý dữ liệu
- Đổi kích thước ảnh về cùng một kích thước (ví dụ 224x224 pixel cho các mô hình CNN phổ biến).
- Chuẩn hóa giá trị pixel (thường chia cho 255 để giá trị nằm trong khoảng 0-1).
- Nếu có thể, áp dụng `augmentation` (kỹ thuật tăng cường dữ liệu) để làm phong phú tập huấn luyện.

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Định nghĩa các tham số hình ảnh
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Tạo đối tượng tăng cường dữ liệu cho huấn luyện
train_datagen = ImageDataGenerator(
    rescale=1./255,           # chuẩn hóa ảnh
    rotation_range=20,        # xoay ảnh ngẫu nhiên
    width_shift_range=0.2,    # dịch chuyển theo chiều ngang
    height_shift_range=0.2,   # dịch chuyển theo chiều dọc
    shear_range=0.2,          # gia công xoắn ảnh
    zoom_range=0.2,           # phóng to/thu nhỏ
    horizontal_flip=True,     # lật ảnh theo chiều ngang
    fill_mode='nearest'       # xử lý điểm ảnh biên
)

# Tạo đối tượng cho tập kiểm tra (không áp dụng augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

# Đường dẫn đến thư mục dữ liệu huấn luyện và testing trên máy của bạn
train_dir = './data/train'
test_dir = './data/test'

# Tạo generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # hoặc 'sparse' nếu nhãn được mã hóa dưới dạng số nguyên
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


# Bước 2: Xây dựng mô hình CNN để phân loại ảnh
- Có thể sẽ xây dựng mô hình từ đầu hoặc sử dụng transfer learning (mô hình tiền huấn luyện như VGG16, ResNet50...)

### Sử dụng `Transfer learning` với mô hình `MobileNetV2`:

In [28]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Tải mô hình MobileNetV2 với trọng số đã được huấn luyện trên ImageNet, không bao gồm phần đầu phân loại
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Đóng băng các tầng của mô hình cơ sở (không huấn luyện lại các weight đã học)
base_model.trainable = False

# Xây dựng phần head cho phân loại
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # chặn quá khớp
# Số lớp đầu ra = số loại chó trong tập dữ liệu (nếu bạn đã kiểm tra, ví dụ giả sử có 120 loại)
num_classes = 120  
predictions = Dense(num_classes, activation='softmax')(x)

# Khởi tạo mô hình hoàn chỉnh
model = Model(inputs=base_model.input, outputs=predictions)

# Biên dịch mô hình
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,411,704 (9.20 MB)

 Trainable params: 153,720 (600.47 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

# Bước 3: Huấn luyện mô hình

In [17]:
epochs = 10  # thay đổi theo số epoch mong muốn

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)

C:\Users\Dell\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 600s 2s/step - accuracy: 0.2551 - loss: 3.3944 - val_accuracy: 0.7367 - val_loss: 0.9377
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 377s 1s/step - accuracy: 0.6054 - loss: 1.4027 - val_accuracy: 0.7639 - val_loss: 0.7702
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 377s 1s/step - accuracy: 0.6458 - loss: 1.1985 - val_accuracy: 0.7682 - val_loss: 0.7489
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 358s 954ms/step - accuracy: 0.6822 - loss: 1.0886 - val_accuracy: 0.7769 - val_loss: 0.7096
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 326s 871ms/step - accuracy: 0.6824 - loss: 1.0784 - val_accuracy: 0.7787 - val_loss: 0.6979
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 361s 963ms/step - accuracy: 0.7054 - loss: 0.9970 - val_accuracy: 0.7864 - val_loss: 0.6985
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 377s 1s/step - accuracy: 0.7026 - loss: 0.9768 - val_accuracy: 0.7740 - val_loss: 0.7188
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 372s 993ms/step - accuracy: 0.7164 - loss: 0.961

In [20]:
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)

269/269 ━━━━━━━━━━━━━━━━━━━━ 89s 329ms/step - accuracy: 0.7833 - loss: 0.7104
Test accuracy: 0.7833333611488342


In [22]:
model.save('dog_classifier_model.h5')

## Bước 4: Fine Tuning mô hình

In [30]:
# Chọn từ tầng nào của base model sẽ mở khóa để cập nhật trọng số
fine_tune_at = 100  # Mở khóa từ tầng thứ 100 trở đi
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in base_model.layers[fine_tune_at:]:
    layer.trainable = True

# Sử dụng learning rate thấp hơn khi fine tuning để tránh thay đổi đột ngột trọng số đã học
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,411,704 (9.20 MB)

 Trainable params: 2,015,160 (7.69 MB)

 Non-trainable params: 396,544 (1.51 MB)

In [ ]:
fine_tune_epochs = 10  # Số epoch fine tuning, thay đổi theo dữ liệu và tài nguyên
total_epochs = epochs + fine_tune_epochs

print("\n--- Fine tuning toàn bộ mô hình ---")
history_fine = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1] + 1,
    validation_data=test_generator
)


--- Fine tuning toàn bộ mô hình ---
Epoch 11/20
